In [15]:
import tflite_runtime.interpreter as tf
from pynq import Overlay, allocate
import numpy as np
import random
import time
import pickle
import matplotlib.pyplot as plt
import random

# Tensorflow Lite Load Model

In [16]:
tflite_interpreter = tf.Interpreter(model_path='model.tflite')
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

# tensor_details = tflite_interpreter.get_tensor_details()

== Input details ==
name: serving_default_input_1:0
shape: [ 1 28 28  1]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.float32'>


# FPGA CNN Overlay Load

In [17]:
overlay = Overlay('./cnn_custom.bit')

In [18]:
dma = overlay.axi_dma_0

# Load Conv Weights

In [19]:
weight = tflite_interpreter.tensor(2)()
weight = np.array(weight * 127.0).astype(np.int16)
print(weight.shape, weight.dtype)
print(max(weight.ravel()), min(weight.ravel()))

OFMCH = weight.shape[0]
IFMCH = weight.shape[3]
KerDim = weight.shape[1]

IFMDim = 1
OFMDim = 1
kernel_val = weight.ravel()
kernel = np.append([1, 0, KerDim, IFMCH, IFMDim, OFMCH, OFMDim], kernel_val)
print(kernel, len(kernel))

in_conv = allocate(shape=(len(kernel),), dtype=np.int16)
out_conv = allocate(shape=(len(kernel),), dtype=np.int16)

np.copyto(in_conv, kernel)

start = time.time()
dma.sendchannel.transfer(in_conv)
dma.recvchannel.transfer(out_conv)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time()
fpga_runtime = end - start
print(fpga_runtime)

print((in_conv == out_conv).all())
in_conv.close()
out_conv.close()

(16, 5, 5, 1) int16
10 -19
[  1   0   5   1   1  16   1  -1   0  -6 -12  -8  -3  -9  -1   0  -5 -13
  -5  -6   4  -4 -10  -1   4   5   3 -14 -16  -3  -3   0  -1 -10   0  -1
   0 -12   0   0   0   0 -17   0  -1   2  -1  -7   0   1   1  -4  -4   0
   0   2  -5  -3   0  -8  -9   1   5   2   0  -8   0  -5  -9   0  -7   1
   0  -1   2 -10   1  -2  -7  -8   0   0  -2 -15   0  -8   0   0   2   0
  -7 -14  -1 -14 -12  -7   0  -6  -3  10  -8  -7  -6   3  -3   3   0 -14
   0  -2  -1   3  -7  -9   2  -8  -5   2  -6   4   0  -1   2 -15  -7   4
   0  -5   0  -5  -4  -5   0   0   1   1   0   0   0   0   0  -4  -5   0
  -6  -1  -1   0 -10   0   0   0  -9   0 -10   0   0  -1   0  -4  -9   0
   1   1   3  -5 -12   0   7  -2  -4  -2  -8   0  -7  -8   0 -10  -8  -4
   1   0  -7  -4  -1  -1  -3   0  -2  -7   0   0   2   0  -2  -9  -5   0
   0   1  -2 -10   0   0   0   1  -6   3 -10  -6   0   0 -16   0   2  -7
   1   1   1   1  -6  -5  -5   0 -15   3 -15   0  -4  -5   1 -11   0   0
 -11   1   1 -15   0  -2

# Load FC Weights

In [20]:
weight = tflite_interpreter.tensor(4)()
weight = np.array(weight * 127.0).astype(np.int16)
print(weight.shape, weight.dtype)
print(max(weight.ravel()), min(weight.ravel()))

OFMCH = weight.shape[0]
IFMCH = weight.shape[1]

KerDim = 1
IFMDim = 1
OFMDim = 1
kernel_val = weight.ravel()
kernel = np.append([2, 0, KerDim, IFMCH, IFMDim, OFMCH, OFMDim], kernel_val)
print(kernel, len(kernel))

in_fc = allocate(shape=(len(kernel),), dtype=np.int16)
out_fc = allocate(shape=(len(kernel),), dtype=np.int16)

np.copyto(in_fc, kernel)

start = time.time()
dma.sendchannel.transfer(in_fc)
dma.recvchannel.transfer(out_fc)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time()
fpga_runtime = end - start
print(fpga_runtime)

print((in_fc == out_fc).all())
in_fc.close()
out_fc.close()

(10, 576) int16
27 -31
[  2   0   1 ...  -6   0 -15] 5767
0.0030667781829833984
True


# Tensorflow Lite Setup

In [21]:
f = open('mnist.pkl', 'rb')
mnist_ds = pickle.load(f, encoding='bytes')
f.close()
(_, _), (x_test, y_test) = mnist_ds

# Load Image from Rand # and forward to DMA

In [22]:
num = 2

in_buffer = allocate(shape=(7 + 28*28,), dtype=np.int16)
out_buffer = allocate(shape=(17,), dtype=np.int16)

img = np.array(x_test[num] / 2).astype(np.int16)
input_val = np.append([0, 1, 1, 1, 28, 10, 1], img.ravel())
np.copyto(in_buffer, input_val)

start = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time()

out = out_buffer[7:]
print(out, y_test[num])

fpga_runtime = end - start
print(fpga_runtime)

in_buffer.close()
out_buffer.close()

[   0 5827    0    0 2774   36  922  571 1914    0] 1
0.04111313819885254


In [23]:
import cnn

conv_w = tflite_interpreter.tensor(2)()
conv_w = np.array(conv_w * 127).astype(np.int8)
print(conv_w.shape, conv_w.dtype, max(conv_w.ravel()), min(conv_w.ravel()))

fc_w = tflite_interpreter.tensor(4)()
fc_w = np.array(fc_w * 127).astype(np.int8)
print(fc_w.shape, fc_w.dtype, max(fc_w.ravel()), min(fc_w.ravel()))


img = np.array(x_test[0])
img = (img / 2).astype(np.int16)
img = np.expand_dims(img, axis=2)
print(img.shape, max(img.ravel()), min(img.ravel()), y_test[0])

cnn = cnn.Cnn()
cnn.dtype = np.int16

start = time.time()

l1 = cnn.conv2d(img, conv_w)
l1b = cnn.relu2d(l1, 3)
l2 = cnn.maxpool2d(l1b, 4)
l3 = cnn.flatten(l2)
l4 = cnn.fc(l3, fc_w)
print(list(l4).index(max(l4)) == y_test[0])
end = time.time()

arm_runtime = end - start
print('Runtime # ', arm_runtime)

(16, 5, 5, 1) int8 10 -19
(10, 576) int8 27 -31
(28, 28, 1) 127 0 7
True
Runtime #  2.378852367401123


In [24]:
in_buffer = allocate(shape=(7 + 28*28,), dtype=np.int16)
out_buffer = allocate(shape=(17,), dtype=np.int16)

acc = 0

start = time.time()
for i in range(len(y_test)):
    img = np.array(x_test[i] / 2).astype(np.int16)
    input_val = np.append([0, 1, 1, 1, 28, 10, 1], img.ravel())
    np.copyto(in_buffer, input_val)

    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    out = out_buffer[7:]
    out = list(out).index(max(out))
    acc += 1 if out == y_test[i] else 0
    print('i = ' + str(i))
    print('accuracy = ' + str(format(acc)))
    print('\n')
end = time.time()
fpga_runtime = end - start
print(fpga_runtime)

in_buffer.close()
out_buffer.close()

i = 0
accuracy = 1


i = 1
accuracy = 2


i = 2
accuracy = 3


i = 3
accuracy = 4


i = 4
accuracy = 5


i = 5
accuracy = 6


i = 6
accuracy = 7


i = 7
accuracy = 8


i = 8
accuracy = 9


i = 9
accuracy = 10


i = 10
accuracy = 11


i = 11
accuracy = 12


i = 12
accuracy = 13


i = 13
accuracy = 14


i = 14
accuracy = 15


i = 15
accuracy = 16


i = 16
accuracy = 17


i = 17
accuracy = 18


i = 18
accuracy = 18


i = 19
accuracy = 19


i = 20
accuracy = 19


i = 21
accuracy = 20


i = 22
accuracy = 20


i = 23
accuracy = 21


i = 24
accuracy = 22


i = 25
accuracy = 23


i = 26
accuracy = 24


i = 27
accuracy = 25


i = 28
accuracy = 26


i = 29
accuracy = 27


i = 30
accuracy = 28


i = 31
accuracy = 29


i = 32
accuracy = 30


i = 33
accuracy = 30


i = 34
accuracy = 31


i = 35
accuracy = 32


i = 36
accuracy = 33


i = 37
accuracy = 34


i = 38
accuracy = 35


i = 39
accuracy = 36


i = 40
accuracy = 37


i = 41
accuracy = 38


i = 42
accuracy = 39


i = 43
accuracy = 40


i = 44


i = 374
accuracy = 360


i = 375
accuracy = 361


i = 376
accuracy = 362


i = 377
accuracy = 363


i = 378
accuracy = 364


i = 379
accuracy = 365


i = 380
accuracy = 366


i = 381
accuracy = 367


i = 382
accuracy = 368


i = 383
accuracy = 369


i = 384
accuracy = 370


i = 385
accuracy = 371


i = 386
accuracy = 372


i = 387
accuracy = 373


i = 388
accuracy = 374


i = 389
accuracy = 375


i = 390
accuracy = 376


i = 391
accuracy = 376


i = 392
accuracy = 377


i = 393
accuracy = 378


i = 394
accuracy = 379


i = 395
accuracy = 380


i = 396
accuracy = 381


i = 397
accuracy = 382


i = 398
accuracy = 383


i = 399
accuracy = 384


i = 400
accuracy = 385


i = 401
accuracy = 386


i = 402
accuracy = 387


i = 403
accuracy = 388


i = 404
accuracy = 389


i = 405
accuracy = 390


i = 406
accuracy = 391


i = 407
accuracy = 392


i = 408
accuracy = 393


i = 409
accuracy = 394


i = 410
accuracy = 395


i = 411
accuracy = 396


i = 412
accuracy = 397


i = 413
accuracy = 398




i = 750
accuracy = 712


i = 751
accuracy = 713


i = 752
accuracy = 714


i = 753
accuracy = 715


i = 754
accuracy = 716


i = 755
accuracy = 717


i = 756
accuracy = 718


i = 757
accuracy = 719


i = 758
accuracy = 720


i = 759
accuracy = 721


i = 760
accuracy = 722


i = 761
accuracy = 723


i = 762
accuracy = 724


i = 763
accuracy = 725


i = 764
accuracy = 726


i = 765
accuracy = 727


i = 766
accuracy = 728


i = 767
accuracy = 729


i = 768
accuracy = 730


i = 769
accuracy = 731


i = 770
accuracy = 732


i = 771
accuracy = 733


i = 772
accuracy = 734


i = 773
accuracy = 735


i = 774
accuracy = 736


i = 775
accuracy = 737


i = 776
accuracy = 738


i = 777
accuracy = 739


i = 778
accuracy = 740


i = 779
accuracy = 741


i = 780
accuracy = 742


i = 781
accuracy = 743


i = 782
accuracy = 744


i = 783
accuracy = 745


i = 784
accuracy = 746


i = 785
accuracy = 747


i = 786
accuracy = 748


i = 787
accuracy = 749


i = 788
accuracy = 750


i = 789
accuracy = 751




i = 1131
accuracy = 1079


i = 1132
accuracy = 1080


i = 1133
accuracy = 1081


i = 1134
accuracy = 1082


i = 1135
accuracy = 1083


i = 1136
accuracy = 1084


i = 1137
accuracy = 1085


i = 1138
accuracy = 1086


i = 1139
accuracy = 1087


i = 1140
accuracy = 1088


i = 1141
accuracy = 1089


i = 1142
accuracy = 1090


i = 1143
accuracy = 1091


i = 1144
accuracy = 1092


i = 1145
accuracy = 1093


i = 1146
accuracy = 1094


i = 1147
accuracy = 1095


i = 1148
accuracy = 1096


i = 1149
accuracy = 1097


i = 1150
accuracy = 1098


i = 1151
accuracy = 1099


i = 1152
accuracy = 1100


i = 1153
accuracy = 1101


i = 1154
accuracy = 1102


i = 1155
accuracy = 1103


i = 1156
accuracy = 1104


i = 1157
accuracy = 1105


i = 1158
accuracy = 1106


i = 1159
accuracy = 1107


i = 1160
accuracy = 1108


i = 1161
accuracy = 1109


i = 1162
accuracy = 1110


i = 1163
accuracy = 1111


i = 1164
accuracy = 1112


i = 1165
accuracy = 1113


i = 1166
accuracy = 1114


i = 1167
accuracy = 1114


i

i = 1502
accuracy = 1428


i = 1503
accuracy = 1429


i = 1504
accuracy = 1430


i = 1505
accuracy = 1431


i = 1506
accuracy = 1432


i = 1507
accuracy = 1433


i = 1508
accuracy = 1434


i = 1509
accuracy = 1435


i = 1510
accuracy = 1436


i = 1511
accuracy = 1437


i = 1512
accuracy = 1438


i = 1513
accuracy = 1439


i = 1514
accuracy = 1440


i = 1515
accuracy = 1441


i = 1516
accuracy = 1442


i = 1517
accuracy = 1443


i = 1518
accuracy = 1444


i = 1519
accuracy = 1445


i = 1520
accuracy = 1446


i = 1521
accuracy = 1447


i = 1522
accuracy = 1448


i = 1523
accuracy = 1448


i = 1524
accuracy = 1449


i = 1525
accuracy = 1450


i = 1526
accuracy = 1451


i = 1527
accuracy = 1451


i = 1528
accuracy = 1452


i = 1529
accuracy = 1453


i = 1530
accuracy = 1453


i = 1531
accuracy = 1454


i = 1532
accuracy = 1455


i = 1533
accuracy = 1456


i = 1534
accuracy = 1457


i = 1535
accuracy = 1458


i = 1536
accuracy = 1459


i = 1537
accuracy = 1460


i = 1538
accuracy = 1461


i

...

i = 8323
accuracy = 7954


i = 8324
accuracy = 7955


i = 8325
accuracy = 7955


i = 8326
accuracy = 7956


i = 8327
accuracy = 7957


i = 8328
accuracy = 7958


i = 8329
accuracy = 7959


i = 8330
accuracy = 7959


i = 8331
accuracy = 7960


i = 8332
accuracy = 7960


i = 8333
accuracy = 7961


i = 8334
accuracy = 7961


i = 8335
accuracy = 7962


i = 8336
accuracy = 7963


i = 8337
accuracy = 7964


i = 8338
accuracy = 7965


i = 8339
accuracy = 7966


i = 8340
accuracy = 7967


i = 8341
accuracy = 7968


i = 8342
accuracy = 7969


i = 8343
accuracy = 7970


i = 8344
accuracy = 7971


i = 8345
accuracy = 7972


i = 8346
accuracy = 7973


i = 8347
accuracy = 7974


i = 8348
accuracy = 7975


i = 8349
accuracy = 7976


i = 8350
accuracy = 7977


i = 8351
accuracy = 7978


i = 8352
accuracy = 7979


i = 8353
accuracy = 7980


i = 8354
accuracy = 7981


i = 8355
accuracy = 7982


i = 8356
accuracy = 7983


i = 8357
accuracy = 7984


i = 8358
accuracy = 7985


i = 8359
accuracy = 7986


i

i = 8704
accuracy = 8321


i = 8705
accuracy = 8322


i = 8706
accuracy = 8323


i = 8707
accuracy = 8324


i = 8708
accuracy = 8325


i = 8709
accuracy = 8326


i = 8710
accuracy = 8327


i = 8711
accuracy = 8328


i = 8712
accuracy = 8329


i = 8713
accuracy = 8330


i = 8714
accuracy = 8331


i = 8715
accuracy = 8332


i = 8716
accuracy = 8333


i = 8717
accuracy = 8334


i = 8718
accuracy = 8335


i = 8719
accuracy = 8336


i = 8720
accuracy = 8337


i = 8721
accuracy = 8338


i = 8722
accuracy = 8339


i = 8723
accuracy = 8340


i = 8724
accuracy = 8341


i = 8725
accuracy = 8342


i = 8726
accuracy = 8343


i = 8727
accuracy = 8344


i = 8728
accuracy = 8345


i = 8729
accuracy = 8346


i = 8730
accuracy = 8347


i = 8731
accuracy = 8348


i = 8732
accuracy = 8349


i = 8733
accuracy = 8350


i = 8734
accuracy = 8351


i = 8735
accuracy = 8352


i = 8736
accuracy = 8353


i = 8737
accuracy = 8354


i = 8738
accuracy = 8355


i = 8739
accuracy = 8356


i = 8740
accuracy = 8357


i

i = 9074
accuracy = 8683


i = 9075
accuracy = 8684


i = 9076
accuracy = 8685


i = 9077
accuracy = 8686


i = 9078
accuracy = 8687


i = 9079
accuracy = 8688


i = 9080
accuracy = 8689


i = 9081
accuracy = 8690


i = 9082
accuracy = 8691


i = 9083
accuracy = 8692


i = 9084
accuracy = 8693


i = 9085
accuracy = 8694


i = 9086
accuracy = 8695


i = 9087
accuracy = 8696


i = 9088
accuracy = 8697


i = 9089
accuracy = 8698


i = 9090
accuracy = 8699


i = 9091
accuracy = 8700


i = 9092
accuracy = 8701


i = 9093
accuracy = 8702


i = 9094
accuracy = 8703


i = 9095
accuracy = 8704


i = 9096
accuracy = 8705


i = 9097
accuracy = 8706


i = 9098
accuracy = 8707


i = 9099
accuracy = 8708


i = 9100
accuracy = 8709


i = 9101
accuracy = 8710


i = 9102
accuracy = 8711


i = 9103
accuracy = 8712


i = 9104
accuracy = 8713


i = 9105
accuracy = 8714


i = 9106
accuracy = 8715


i = 9107
accuracy = 8716


i = 9108
accuracy = 8717


i = 9109
accuracy = 8718


i = 9110
accuracy = 8719


i

i = 9447
accuracy = 9054


i = 9448
accuracy = 9055


i = 9449
accuracy = 9056


i = 9450
accuracy = 9057


i = 9451
accuracy = 9058


i = 9452
accuracy = 9059


i = 9453
accuracy = 9060


i = 9454
accuracy = 9061


i = 9455
accuracy = 9062


i = 9456
accuracy = 9063


i = 9457
accuracy = 9064


i = 9458
accuracy = 9065


i = 9459
accuracy = 9066


i = 9460
accuracy = 9067


i = 9461
accuracy = 9068


i = 9462
accuracy = 9069


i = 9463
accuracy = 9070


i = 9464
accuracy = 9071


i = 9465
accuracy = 9072


i = 9466
accuracy = 9073


i = 9467
accuracy = 9074


i = 9468
accuracy = 9075


i = 9469
accuracy = 9075


i = 9470
accuracy = 9076


i = 9471
accuracy = 9077


i = 9472
accuracy = 9078


i = 9473
accuracy = 9079


i = 9474
accuracy = 9080


i = 9475
accuracy = 9081


i = 9476
accuracy = 9082


i = 9477
accuracy = 9083


i = 9478
accuracy = 9084


i = 9479
accuracy = 9085


i = 9480
accuracy = 9086


i = 9481
accuracy = 9087


i = 9482
accuracy = 9088


i = 9483
accuracy = 9089


i

i = 9827
accuracy = 9410


i = 9828
accuracy = 9411


i = 9829
accuracy = 9412


i = 9830
accuracy = 9413


i = 9831
accuracy = 9414


i = 9832
accuracy = 9415


i = 9833
accuracy = 9416


i = 9834
accuracy = 9417


i = 9835
accuracy = 9418


i = 9836
accuracy = 9419


i = 9837
accuracy = 9420


i = 9838
accuracy = 9421


i = 9839
accuracy = 9422


i = 9840
accuracy = 9423


i = 9841
accuracy = 9424


i = 9842
accuracy = 9425


i = 9843
accuracy = 9426


i = 9844
accuracy = 9427


i = 9845
accuracy = 9428


i = 9846
accuracy = 9429


i = 9847
accuracy = 9430


i = 9848
accuracy = 9431


i = 9849
accuracy = 9432


i = 9850
accuracy = 9433


i = 9851
accuracy = 9434


i = 9852
accuracy = 9435


i = 9853
accuracy = 9436


i = 9854
accuracy = 9437


i = 9855
accuracy = 9438


i = 9856
accuracy = 9439


i = 9857
accuracy = 9440


i = 9858
accuracy = 9441


i = 9859
accuracy = 9442


i = 9860
accuracy = 9443


i = 9861
accuracy = 9444


i = 9862
accuracy = 9445


i = 9863
accuracy = 9446


i

In [25]:
len(y_test)

10000

In [26]:
import cnn

conv_w = tflite_interpreter.tensor(2)()
conv_w = np.array(conv_w * 127).astype(np.int8)

fc_w = tflite_interpreter.tensor(4)()
fc_w = np.array(fc_w * 127).astype(np.int8)


cnn = cnn.Cnn()
cnn.dtype = np.int16

start = time.time()
acc_arm = 0

for i in range(len(y_test)):
    img = np.array(x_test[i] / 2).astype(np.int8)
    img = np.expand_dims(img, 2)
    l1 = cnn.conv2d(img, conv_w)
    l1b = cnn.relu2d(l1, 3)
    l2 = cnn.maxpool2d(l1b, 4)
    l3 = cnn.flatten(l2)
    l4 = cnn.fc(l3, fc_w)
    acc_arm += 1 if y_test[i] == list(l4).index(max(l4)) else 0
    print('accuracy = ' + str(format(acc_arm)))

end = time.time()
arm_runtime = end - start
print('Runtime # ', arm_runtime)

accuracy = 1
accuracy = 2
accuracy = 3
accuracy = 4
accuracy = 5
accuracy = 6
accuracy = 7
accuracy = 8
accuracy = 9
accuracy = 10
accuracy = 11
accuracy = 12
accuracy = 13
accuracy = 14
accuracy = 15
accuracy = 16
accuracy = 17
accuracy = 18
accuracy = 18
accuracy = 19
accuracy = 19
accuracy = 20
accuracy = 20
accuracy = 21
accuracy = 22
accuracy = 23
accuracy = 24
accuracy = 25
accuracy = 26
accuracy = 27
accuracy = 28
accuracy = 29
accuracy = 30
accuracy = 30
accuracy = 31
accuracy = 32
accuracy = 33
accuracy = 34
accuracy = 35
accuracy = 36
accuracy = 37
accuracy = 38
accuracy = 39
accuracy = 40
accuracy = 41
accuracy = 42
accuracy = 43
accuracy = 44
accuracy = 45
accuracy = 46
accuracy = 47
accuracy = 48
accuracy = 49
accuracy = 50
accuracy = 51
accuracy = 52
accuracy = 53
accuracy = 54
accuracy = 55
accuracy = 56
accuracy = 57
accuracy = 58
accuracy = 58
accuracy = 59
accuracy = 60
accuracy = 61
accuracy = 62
accuracy = 63
accuracy = 64
accuracy = 65
accuracy = 66
accuracy = 67
a

accuracy = 528
accuracy = 529
accuracy = 530
accuracy = 530
accuracy = 531
accuracy = 532
accuracy = 533
accuracy = 534
accuracy = 535
accuracy = 536
accuracy = 537
accuracy = 538
accuracy = 539
accuracy = 540
accuracy = 541
accuracy = 542
accuracy = 543
accuracy = 544
accuracy = 545
accuracy = 546
accuracy = 547
accuracy = 548
accuracy = 549
accuracy = 550
accuracy = 551
accuracy = 552
accuracy = 553
accuracy = 554
accuracy = 554
accuracy = 555
accuracy = 556
accuracy = 557
accuracy = 558
accuracy = 559
accuracy = 560
accuracy = 561
accuracy = 562
accuracy = 562
accuracy = 563
accuracy = 564
accuracy = 565
accuracy = 566
accuracy = 566
accuracy = 567
accuracy = 568
accuracy = 569
accuracy = 570
accuracy = 571
accuracy = 572
accuracy = 573
accuracy = 574
accuracy = 575
accuracy = 576
accuracy = 577
accuracy = 578
accuracy = 579
accuracy = 580
accuracy = 581
accuracy = 582
accuracy = 583
accuracy = 584
accuracy = 585
accuracy = 586
accuracy = 587
accuracy = 588
accuracy = 589
accuracy =

accuracy = 1047
accuracy = 1048
accuracy = 1049
accuracy = 1050
accuracy = 1051
accuracy = 1052
accuracy = 1053
accuracy = 1054
accuracy = 1055
accuracy = 1055
accuracy = 1056
accuracy = 1057
accuracy = 1058
accuracy = 1059
accuracy = 1060
accuracy = 1061
accuracy = 1062
accuracy = 1063
accuracy = 1064
accuracy = 1065
accuracy = 1066
accuracy = 1067
accuracy = 1068
accuracy = 1069
accuracy = 1070
accuracy = 1071
accuracy = 1071
accuracy = 1072
accuracy = 1073
accuracy = 1074
accuracy = 1075
accuracy = 1076
accuracy = 1077
accuracy = 1078
accuracy = 1079
accuracy = 1080
accuracy = 1081
accuracy = 1082
accuracy = 1083
accuracy = 1084
accuracy = 1085
accuracy = 1086
accuracy = 1087
accuracy = 1088
accuracy = 1089
accuracy = 1090
accuracy = 1091
accuracy = 1092
accuracy = 1093
accuracy = 1094
accuracy = 1095
accuracy = 1096
accuracy = 1097
accuracy = 1098
accuracy = 1099
accuracy = 1100
accuracy = 1101
accuracy = 1102
accuracy = 1103
accuracy = 1104
accuracy = 1105
accuracy = 1106
accuracy

accuracy = 1527
accuracy = 1528
accuracy = 1529
accuracy = 1530
accuracy = 1531
accuracy = 1532
accuracy = 1533
accuracy = 1534
accuracy = 1535
accuracy = 1536
accuracy = 1537
accuracy = 1538
accuracy = 1539
accuracy = 1540
accuracy = 1541
accuracy = 1542
accuracy = 1543
accuracy = 1544
accuracy = 1545
accuracy = 1546
accuracy = 1547
accuracy = 1548
accuracy = 1549
accuracy = 1550
accuracy = 1551
accuracy = 1552
accuracy = 1553
accuracy = 1554
accuracy = 1555
accuracy = 1556
accuracy = 1557
accuracy = 1558
accuracy = 1559
accuracy = 1560
accuracy = 1561
accuracy = 1562
accuracy = 1563
accuracy = 1564
accuracy = 1565
accuracy = 1566
accuracy = 1567
accuracy = 1568
accuracy = 1569
accuracy = 1570
accuracy = 1571
accuracy = 1572
accuracy = 1573
accuracy = 1574
accuracy = 1575
accuracy = 1576
accuracy = 1577
accuracy = 1578
accuracy = 1579
accuracy = 1580
accuracy = 1581
accuracy = 1582
accuracy = 1583
accuracy = 1584
accuracy = 1585
accuracy = 1586
accuracy = 1587
accuracy = 1588
accuracy

accuracy = 2016
accuracy = 2017
accuracy = 2018
accuracy = 2019
accuracy = 2020
accuracy = 2020
accuracy = 2020
accuracy = 2021
accuracy = 2022
accuracy = 2023
accuracy = 2024
accuracy = 2024
accuracy = 2025
accuracy = 2026
accuracy = 2027
accuracy = 2028
accuracy = 2029
accuracy = 2030
accuracy = 2031
accuracy = 2032
accuracy = 2033
accuracy = 2034
accuracy = 2035
accuracy = 2036
accuracy = 2037
accuracy = 2037
accuracy = 2038
accuracy = 2039
accuracy = 2040
accuracy = 2041
accuracy = 2042
accuracy = 2043
accuracy = 2044
accuracy = 2045
accuracy = 2046
accuracy = 2047
accuracy = 2048
accuracy = 2049
accuracy = 2050
accuracy = 2051
accuracy = 2052
accuracy = 2053
accuracy = 2054
accuracy = 2055
accuracy = 2056
accuracy = 2057
accuracy = 2058
accuracy = 2059
accuracy = 2060
accuracy = 2061
accuracy = 2062
accuracy = 2063
accuracy = 2064
accuracy = 2065
accuracy = 2066
accuracy = 2066
accuracy = 2067
accuracy = 2068
accuracy = 2068
accuracy = 2069
accuracy = 2070
accuracy = 2071
accuracy

...

accuracy = 7412
accuracy = 7413
accuracy = 7414
accuracy = 7415
accuracy = 7416
accuracy = 7417
accuracy = 7418
accuracy = 7419
accuracy = 7420
accuracy = 7421
accuracy = 7422
accuracy = 7423
accuracy = 7424
accuracy = 7425
accuracy = 7426
accuracy = 7427
accuracy = 7428
accuracy = 7429
accuracy = 7430
accuracy = 7431
accuracy = 7432
accuracy = 7433
accuracy = 7434
accuracy = 7435
accuracy = 7436
accuracy = 7437
accuracy = 7438
accuracy = 7439
accuracy = 7440
accuracy = 7441
accuracy = 7442
accuracy = 7443
accuracy = 7444
accuracy = 7445
accuracy = 7446
accuracy = 7447
accuracy = 7448
accuracy = 7449
accuracy = 7450
accuracy = 7451
accuracy = 7452
accuracy = 7453
accuracy = 7454
accuracy = 7455
accuracy = 7456
accuracy = 7457
accuracy = 7458
accuracy = 7459
accuracy = 7460
accuracy = 7461
accuracy = 7462
accuracy = 7463
accuracy = 7464
accuracy = 7465
accuracy = 7466
accuracy = 7467
accuracy = 7468
accuracy = 7469
accuracy = 7470
accuracy = 7471
accuracy = 7472
accuracy = 7473
accuracy

accuracy = 7910
accuracy = 7911
accuracy = 7912
accuracy = 7913
accuracy = 7914
accuracy = 7915
accuracy = 7916
accuracy = 7916
accuracy = 7917
accuracy = 7918
accuracy = 7919
accuracy = 7920
accuracy = 7921
accuracy = 7922
accuracy = 7923
accuracy = 7924
accuracy = 7925
accuracy = 7926
accuracy = 7927
accuracy = 7928
accuracy = 7929
accuracy = 7930
accuracy = 7931
accuracy = 7932
accuracy = 7933
accuracy = 7934
accuracy = 7935
accuracy = 7936
accuracy = 7937
accuracy = 7938
accuracy = 7939
accuracy = 7940
accuracy = 7941
accuracy = 7942
accuracy = 7943
accuracy = 7944
accuracy = 7944
accuracy = 7945
accuracy = 7946
accuracy = 7947
accuracy = 7948
accuracy = 7949
accuracy = 7949
accuracy = 7950
accuracy = 7951
accuracy = 7951
accuracy = 7952
accuracy = 7953
accuracy = 7954
accuracy = 7955
accuracy = 7955
accuracy = 7956
accuracy = 7956
accuracy = 7957
accuracy = 7957
accuracy = 7958
accuracy = 7959
accuracy = 7960
accuracy = 7961
accuracy = 7962
accuracy = 7963
accuracy = 7964
accuracy

accuracy = 8405
accuracy = 8406
accuracy = 8407
accuracy = 8408
accuracy = 8409
accuracy = 8410
accuracy = 8411
accuracy = 8412
accuracy = 8413
accuracy = 8414
accuracy = 8415
accuracy = 8416
accuracy = 8417
accuracy = 8417
accuracy = 8418
accuracy = 8419
accuracy = 8420
accuracy = 8421
accuracy = 8422
accuracy = 8423
accuracy = 8424
accuracy = 8425
accuracy = 8426
accuracy = 8427
accuracy = 8428
accuracy = 8429
accuracy = 8430
accuracy = 8431
accuracy = 8432
accuracy = 8433
accuracy = 8434
accuracy = 8435
accuracy = 8436
accuracy = 8437
accuracy = 8438
accuracy = 8439
accuracy = 8440
accuracy = 8441
accuracy = 8442
accuracy = 8443
accuracy = 8444
accuracy = 8445
accuracy = 8446
accuracy = 8447
accuracy = 8448
accuracy = 8449
accuracy = 8450
accuracy = 8451
accuracy = 8452
accuracy = 8453
accuracy = 8454
accuracy = 8455
accuracy = 8456
accuracy = 8457
accuracy = 8458
accuracy = 8459
accuracy = 8460
accuracy = 8461
accuracy = 8462
accuracy = 8463
accuracy = 8464
accuracy = 8465
accuracy

accuracy = 8907
accuracy = 8908
accuracy = 8909
accuracy = 8910
accuracy = 8911
accuracy = 8912
accuracy = 8913
accuracy = 8914
accuracy = 8915
accuracy = 8916
accuracy = 8917
accuracy = 8918
accuracy = 8919
accuracy = 8920
accuracy = 8921
accuracy = 8922
accuracy = 8923
accuracy = 8924
accuracy = 8925
accuracy = 8926
accuracy = 8927
accuracy = 8928
accuracy = 8929
accuracy = 8930
accuracy = 8931
accuracy = 8932
accuracy = 8933
accuracy = 8934
accuracy = 8935
accuracy = 8936
accuracy = 8937
accuracy = 8938
accuracy = 8939
accuracy = 8940
accuracy = 8941
accuracy = 8942
accuracy = 8943
accuracy = 8944
accuracy = 8945
accuracy = 8946
accuracy = 8947
accuracy = 8948
accuracy = 8949
accuracy = 8950
accuracy = 8951
accuracy = 8952
accuracy = 8953
accuracy = 8954
accuracy = 8955
accuracy = 8956
accuracy = 8957
accuracy = 8958
accuracy = 8959
accuracy = 8960
accuracy = 8961
accuracy = 8962
accuracy = 8963
accuracy = 8964
accuracy = 8965
accuracy = 8966
accuracy = 8967
accuracy = 8968
accuracy

accuracy = 9396
accuracy = 9397
accuracy = 9398
accuracy = 9399
accuracy = 9400
accuracy = 9401
accuracy = 9402
accuracy = 9403
accuracy = 9404
accuracy = 9405
accuracy = 9406
accuracy = 9407
accuracy = 9408
accuracy = 9409
accuracy = 9410
accuracy = 9411
accuracy = 9412
accuracy = 9413
accuracy = 9414
accuracy = 9415
accuracy = 9416
accuracy = 9417
accuracy = 9418
accuracy = 9419
accuracy = 9420
accuracy = 9421
accuracy = 9422
accuracy = 9423
accuracy = 9424
accuracy = 9425
accuracy = 9426
accuracy = 9427
accuracy = 9428
accuracy = 9429
accuracy = 9430
accuracy = 9431
accuracy = 9432
accuracy = 9433
accuracy = 9434
accuracy = 9435
accuracy = 9436
accuracy = 9437
accuracy = 9438
accuracy = 9439
accuracy = 9440
accuracy = 9441
accuracy = 9442
accuracy = 9443
accuracy = 9444
accuracy = 9445
accuracy = 9446
accuracy = 9447
accuracy = 9448
accuracy = 9449
accuracy = 9450
accuracy = 9451
accuracy = 9452
accuracy = 9453
accuracy = 9454
accuracy = 9455
accuracy = 9455
accuracy = 9456
accuracy

In [28]:
acc_rate = arm_runtime / fpga_runtime
print('ARM Run Time # 10000 = ', arm_runtime)
print('FPGA Run Time # 10000 = ', fpga_runtime)
print('Total Acceleration', acc_rate)
print('Accuracy loss (Negative means accuracy increase) = ', (acc_arm - acc) / 100, '%')

ARM Run Time # 10000 =  24393.239780664444
FPGA Run Time # 10000 =  22.37247061729431
Total Acceleration 1090.323916295952
Accuracy loss (Negative means accuracy increase) =  -0.06 %
